<a href="https://colab.research.google.com/github/JYYCaN/Hugo-stack-theme-mod/blob/master/test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip list | grep mmcv

mmcv                                  2.2.0


In [2]:
!pip install mmcv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 479.1/479.1 kB 13.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 452.7/452.7 kB 33.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 20.3 MB/s eta 0:00:00
  Created wheel for mmcv: filename=mmcv-2.2.0-cp311-cp311-linux_x86_64.whl size=38326997 sha256=c3573ac1560ee7aec82809cc09ed307761a087cd82ef5814e029a3f32aacdf2c
  Stored in directory: /root/.cache/pip/wheels/52/4f/8c/3145c780918db1a98c6c7a009a92bb1fadf2217118b25c4d53
Successfully built mmcv


In [5]:
import torch
from mmcv.ops import modulated_deform_conv2d as mmcv_modulated_deform_conv2d
import numpy as np

def create_single_cpu_tensor(item, minvalue, maxvalue):
    dtype = item[0]
    format1 = item[1]
    shape = item[2]
    input1 = np.random.uniform(minvalue, maxvalue, shape).astype(dtype)
    return torch.from_numpy(input1).cuda()

def get_fwd_golden(x, offset, mask, weight, groups, grad_out):
    x_cpu = x.detach().clone().cuda()
    offset_cpu = offset.detach().clone().cuda()
    mask_cpu = mask.detach().clone().cuda()
    weight_cpu = weight.detach().clone().cuda()
    x_cpu.grad, offset_cpu.grad, mask_cpu.grad, weight_cpu.grad = None, None, None, None

    x_cpu.requires_grad = True
    offset_cpu.requires_grad = True
    mask_cpu.requires_grad = True
    weight_cpu.requires_grad = True

    out = mmcv_modulated_deform_conv2d(x_cpu, offset_cpu, mask_cpu, weight_cpu, None, 1, 1, 1, groups)
    out.backward(grad_out, retain_graph=True)

    return x_cpu.grad, offset_cpu.grad, mask_cpu.grad, weight_cpu.grad

def gen_bevformer_data():
    cases = [
                [6, 512, 512, 3, 29, 50, 29, 50, 1],
                [6, 256, 256, 3, 58, 100, 58, 100, 1],
                [12, 512, 512, 3, 29, 50, 29, 50, 1],
                [12, 256, 256, 3, 58, 100, 58, 100, 1],
            ]

    for i, case in enumerate(cases):
        N, cIn, cOut, K, hIn, wIn, hOut, wOut, groups = case

        x = create_single_cpu_tensor([np.float32, 0, (N, cIn, hIn, wIn)], -5, 5)
        offset = create_single_cpu_tensor([np.float32, 0, (N, 2 * K * K, hOut, wOut)], -2, 2)
        mask = create_single_cpu_tensor([np.float32, 0, (N, K * K, hOut, wOut)], -5, 5)
        weight = create_single_cpu_tensor([np.float32, 0, (cOut, cIn // groups, K, K)], -5, 5) * 0.001
        grad_out = create_single_cpu_tensor([np.float32, 0, (N, cIn, hOut, wOut)], -5, 5)

        x_grad, offset_grad, mask_grad, weight_grad = get_fwd_golden(x, offset, mask, weight, groups, grad_out)

        data = {
            "x": x,
            "offset": offset,
            "mask": mask,
            "weight": weight,
            "grad_out": grad_out,
            "x_grad": x_grad,
            "offset_grad": offset_grad,
            "mask_grad": mask_grad,
            "weight_grad": weight_grad,
            "attr": [N, cIn, cOut, K, hIn, wIn, hOut, wOut, groups],
        }

        torch.save(data, f"./test_bevformer_model_case_{i}.pt")

gen_bevformer_data()